In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
from auroraPSI.itx_adapter import ItxAdapter
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

## Data loading

We load the original Krakow data in format itx, we resample at different resolutions

In [ ]:
itx_adapter = ItxAdapter.read_file("../data/observations/KRK_input_10min.itx")

krakow_dict = itx_adapter.to_pandas()
# krakow_df = pd.concat([krakow_dict['data'], krakow_dict['errors']])
krakow_df = krakow_dict['data']


In [ ]:
krakow_df.drop(['12.0','37.0'],axis=1,inplace=True)

krakow_df_10min = krakow_df.resample("10min").mean()
krakow_df_30min = krakow_df.resample("30min").mean()
krakow_df_1h = krakow_df.resample("1H").mean()
krakow_df_1d = krakow_df.resample("1D").mean()
krakow_df_1d

In [ ]:
errors = krakow_dict['errors']
errors.drop(['12.0_err','37.0_err'],axis=1,inplace=True)
errors_10min = errors.resample("10min").mean()
values = krakow_df_10min['41.0']
X = krakow_df_10min.values[~np.isnan(values)]
X = X
E = errors_10min.values[~np.isnan(values)]

In [ ]:
nt, nx = X.shape # time, compounds
k = 10 #sources 10 to 20

nt, nx

# Define some functions

In [ ]:
from auroraPSI.NMFEstimator import NMFEstimator

In [ ]:
from auroraPSI.nmf_updates import create_laplacian_matrix
L = create_laplacian_matrix(10).todense()
# np.linalg.eigvals(L)
# L

In [ ]:

plt.imshow(X[:200])
plt.colorbar()
np.min(X)

In [ ]:
plt.imshow(X[:200]<0)

In [ ]:

est = NMFEstimator(n_components=k
             , init='random', tol=1e-4, max_iter=200,
                 random_state=None, verbose=1, debug=True, lambda_L=0,
                shift=shift, eval_print=10, force_simplex=True)

In [ ]:
Xnew = X.copy()
Xnew[X<0] = 0
est.fit(Xnew)

In [ ]:
F = est.F_
G = est.G_
loss = est.get_losses()

In [ ]:
np.sum(G, axis=0)

In [ ]:
import re
mark_space = 20
fontsize = 15
marker_list = ["-o","-s","->","-<","-^","-v","-d"]


fig, axes = plt.subplots(1, 2, figsize=(15, 5))

names = list(loss.dtype.names)
for j,name in enumerate(names) :
    if re.match(r".*(loss)",name) : 
        axes[0].plot(loss[name],marker_list[j%len(marker_list)],markersize=3.5,label = name,markevery = mark_space,linewidth = 2)
        axes[0].set_yscale("log")
        axes[0].legend()
        axes[0].set_xlabel("number of iterations")
    elif re.match(r"^(rel)",name) : 
        axes[1].plot(loss[name],marker_list[j%len(marker_list)],markersize=3.5,label = name,markevery = mark_space,linewidth = 2)
        axes[1].legend()
        axes[1].set_xlabel("number of iterations")
    # elif re.match(r"^(ang)",name) :
    #     axes[2].plot(loss[name],marker_list[j%len(marker_list)],markersize=3.5,label = name,markevery = mark_space,linewidth = 2)
    #     axes[2].legend()
    #     axes[2].set_xlabel("number of iterations")
    # elif re.match(r"^(mse)",name) :
    #     axes[3].plot(loss[name],marker_list[j%len(marker_list)],markersize=3.5,label = name,markevery = mark_space,linewidth = 2)
    #     axes[3].legend()
    #     axes[3].set_xlabel("number of iterations")

cols = ["Losses", "Evolution of A and P","Angles","MSE"]

for ax, col in zip(axes, cols):
    ax.set_title(col, fontsize=fontsize)

fig.tight_layout()

In [ ]:
plt.plot(F);

In [ ]:
plt.imshow(G)
plt.colorbar()